In [1]:
import os
import numpy as np
import mxnet as mx
from model.simple_stack import SimpleStack
from utils import check_dir
from memory import Memory
from environments.SimpleEnv import SimpleEnv
from utils import create_input, translate_state
from evaluation import evaluate
from mxnet import gluon, nd, autograd

In [2]:
# training cases
order = "MXNET"
# batch size
batch_size = 512
# agent view
agent_view = 7
map_size = 20
# action max
action_max = 3
# learning rate
model_save = "./model_save/"
lr = 0.001
num_episode = 1000000
# start play
replay_start = 10000
# update step
update_step = 1000
# gamma in q-loss calculation
gamma = 0.99
# memory pool size
memory_length = 100000
# file to save train log
summary = "./{}_Reward.csv".format(order)
eval_statistics = "./{}_CSV.csv".format(order)
# the number of step it take to linearly anneal the epsilon to it min value
annealing_end = 200000
# min level of stochastically of policy (epsilon)-greedy
epsilon_min = 0.2
# temporary files
temporary_model = "./{}/{}.params".format(model_save, order)
temporary_pool = "./{}/{}.pool".format(model_save, order)

In [3]:
if os.path.exists(summary):
    os.remove(summary)
ctx = mx.gpu()
for i in ["model_save", "data_save"]:
    check_dir(i)

In [4]:
# build models
online_model = SimpleStack()
offline_model = SimpleStack()
online_model.collect_params().initialize(mx.init.MSRAPrelu(), ctx=ctx)
offline_model.collect_params().initialize(mx.init.MSRAPrelu(), ctx=ctx)
offline_model.collect_params().zero_grad()

[01:07:48] ../src/base.cc:80: cuDNN lib mismatch: linked-against version 8101 != compiled-against version 8100.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [5]:
# create env
env = SimpleEnv(display=False, agent_view=agent_view, map_size=map_size)
env.reset_env()
memory_pool = Memory(memory_length)
annealing = 0
total_reward = np.zeros(num_episode)
eval_result = []
loss_func = gluon.loss.L2Loss()
trainer = gluon.Trainer(offline_model.collect_params(), 'adam', {'learning_rate': lr})

In [6]:
_epoch = 0

In [ ]:
_print = True
_last_dr_50 = 0
for epoch in range(num_episode):
    env.reset_env()
    finish = 0
    cum_clipped_dr = 0
    if epoch == 51:
        print("Model Structure: ")
        print(offline_model)
    if sum(env.step_count) > replay_start and _print:
        print('annealing and learning are started')
        _print = False
    while not finish:
        if sum(env.step_count) > replay_start:
            annealing += 1
        eps = np.maximum(1 - sum(env.step_count) / annealing_end, epsilon_min)
        if np.random.random() < eps:
            by = "Random"
            action = np.random.randint(0, action_max)
        else:
            by = "Model"
            data = create_input([translate_state(env.map.state())])
            data = [nd.array(i, ctx=ctx) for i in data]
            action = offline_model(data)
            action = int(nd.argmax(action, axis=1).asnumpy()[0])
        old, new, reward_get, finish = env.step(action)
        memory_pool.add(old, new, action, reward_get, finish)
        if finish and epoch > 50:
            cum_clipped_dr += env.detect_rate[-1]
            dr_50 = float(np.mean(env.detect_rate[-50:]))
            dr_all = float(np.mean(env.detect_rate))
            if epoch % 50 == 0:
                text = "DR: %f(50), %f(all), eps: %f" % (dr_50, dr_all, eps)
                print(text)
                with open(summary, "a") as f:
                    f.writelines(text + "\n")
            if epoch % 100 == 0 and annealing > replay_start:
                eval_result.extend(evaluate(ctx, offline_model, env, 5))
            # save model and replace online model each update_step
            if annealing > replay_start and annealing % update_step == 0:
                if dr_50 >= _last_dr_50:
                    _last_dr_50 = dr_50
                    offline_model.save_parameters(temporary_model)
                    online_model.load_parameters(temporary_model, ctx)
    #  train every 2 epoch
    if annealing > replay_start and epoch % 2 == 0:
        # Sample random mini batch of transitions
        if len(memory_pool.memory) > batch_size:
            bz = batch_size
        else:
            bz = len(memory_pool.memory)
        for_train = memory_pool.next_batch(bz)
        with autograd.record(train_mode=True):
            _state =[nd.array(i, ctx=ctx) for i in for_train["state"]]
            _state_next = [nd.array(i, ctx=ctx) for i in for_train["state_next"]]
            _finish = nd.array(for_train["finish"], ctx=ctx)
            _action = nd.array(for_train["action"], ctx=ctx)
            _reward = nd.array(for_train["reward"], ctx=ctx)
            q_sp = nd.max(online_model(_state_next), axis=1)
            q_sp = q_sp * (nd.ones(bz, ctx=ctx) - _finish)
            q_s_array = offline_model(_state)
            q_s = nd.pick(q_s_array, _action, 1)
            loss = nd.mean(loss_func(q_s, (_reward + gamma * q_sp)))
        loss.backward()
        trainer.step(bz)
    total_reward[int(epoch) - 1] = cum_clipped_dr

[01:07:52] ../src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:97: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)


Model Structure: 
SimpleStack(
  (view): Sequential(
    (0): Conv2D(2 -> 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2D(256 -> 128, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (2): Conv2D(128 -> 128, kernel_size=(2, 2), stride=(1, 1), bias=False)
  )
  (map): Sequential(
    (0): Conv2D(3 -> 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2D(256 -> 256, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (2): Conv2D(256 -> 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (3): Conv2D(128 -> 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (4): Conv2D(128 -> 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (5): Conv2D(128 -> 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  )
  (decision_making): Sequential(
    (0): Dense(16001 -> 1024, Activation(sigmoid))
    (1): Dense(1024 -> 64, Activation(sigmoid))
    (2): Dense(64 -> 3, Activation(sigmoid))
  )
)
DR: 0.085543(50), 0.087625(all), eps: 0.950000
annealing and lea

DR: 0.497640(50), 0.425078(all), eps: 0.200000
DR: 0.566168(50), 0.426256(all), eps: 0.200000
DR: 0.518837(50), 0.426815(all), eps: 0.200000
DR: 0.460053(50), 0.426967(all), eps: 0.200000
DR: 0.363131(50), 0.426586(all), eps: 0.200000
DR: 0.514565(50), 0.426905(all), eps: 0.200000
DR: 0.560272(50), 0.427691(all), eps: 0.200000
DR: 0.468656(50), 0.427922(all), eps: 0.200000
DR: 0.484464(50), 0.428251(all), eps: 0.200000
DR: 0.500106(50), 0.428483(all), eps: 0.200000
DR: 0.514930(50), 0.428980(all), eps: 0.200000
DR: 0.475443(50), 0.429450(all), eps: 0.200000
DR: 0.397558(50), 0.429269(all), eps: 0.200000
DR: 0.468730(50), 0.429381(all), eps: 0.200000
DR: 0.437668(50), 0.429427(all), eps: 0.200000
DR: 0.499910(50), 0.429818(all), eps: 0.200000
DR: 0.384493(50), 0.429566(all), eps: 0.200000
DR: 0.436397(50), 0.429714(all), eps: 0.200000
DR: 0.653182(50), 0.430939(all), eps: 0.200000
DR: 0.594858(50), 0.431874(all), eps: 0.200000
DR: 0.566009(50), 0.432601(all), eps: 0.200000
DR: 0.499066(

DR: 0.535340(50), 0.450274(all), eps: 0.200000
DR: 0.489264(50), 0.450386(all), eps: 0.200000
DR: 0.332597(50), 0.450023(all), eps: 0.200000
DR: 0.432084(50), 0.449972(all), eps: 0.200000
DR: 0.459170(50), 0.450038(all), eps: 0.200000
DR: 0.467825(50), 0.450088(all), eps: 0.200000
DR: 0.510797(50), 0.450251(all), eps: 0.200000
DR: 0.553770(50), 0.450542(all), eps: 0.200000
DR: 0.469503(50), 0.450658(all), eps: 0.200000
DR: 0.426420(50), 0.450590(all), eps: 0.200000
DR: 0.518107(50), 0.450768(all), eps: 0.200000
DR: 0.478397(50), 0.450845(all), eps: 0.200000
DR: 0.461167(50), 0.450793(all), eps: 0.200000
DR: 0.402347(50), 0.450659(all), eps: 0.200000
DR: 0.507893(50), 0.450920(all), eps: 0.200000
DR: 0.478756(50), 0.450997(all), eps: 0.200000
DR: 0.443653(50), 0.450963(all), eps: 0.200000
DR: 0.565228(50), 0.451276(all), eps: 0.200000
DR: 0.479867(50), 0.451347(all), eps: 0.200000
DR: 0.513330(50), 0.451516(all), eps: 0.200000
DR: 0.445174(50), 0.451534(all), eps: 0.200000
DR: 0.496981(